In [22]:
import findspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, col
from pyspark.sql.types import StringType, FloatType, StructType

In [23]:
findspark.init()

spark = SparkSession.builder.master("local").appName("ESTBAN").getOrCreate()

In [42]:
df = (
    spark.read.option("header", True)
    .option("inferSchema", True)
    .option("sep", ";")
    .csv("estban_csv")
)

In [43]:
df = df.withColumn("ano", substring(col("#DATA_BASE"), 1, 4))
df = df.withColumn("mes", substring(col("#DATA_BASE"), 5, 2))

In [44]:
df = df.drop(col("#DATA_BASE"))

In [45]:
import json

In [46]:
with open("columns.json", "r") as f:
    columns = json.load(f)

In [47]:
for index, column in enumerate(df.columns):
    df = df.withColumnRenamed(column, columns[index]["name"])

In [48]:
df.schema

StructType([StructField('uf', StringType(), True), StructField('cod_mun', StringType(), True), StructField('municipio', StringType(), True), StructField('cnpj', StringType(), True), StructField('nome_instituicao', StringType(), True), StructField('cnpj_agencia', StringType(), True), StructField('110', StringType(), True), StructField('111', StringType(), True), StructField('112', StringType(), True), StructField('113', StringType(), True), StructField('114', StringType(), True), StructField('120', StringType(), True), StructField('130', StringType(), True), StructField('140', StringType(), True), StructField('141_142', StringType(), True), StructField('143', StringType(), True), StructField('144_145_146_147_152', StringType(), True), StructField('153', StringType(), True), StructField('158', StringType(), True), StructField('160', StringType(), True), StructField('161', StringType(), True), StructField('162', StringType(), True), StructField('163', StringType(), True), StructField('164

In [49]:
def _cast_json(df, columns):

        root_spark_schema = {"fields": [], "type": "struct"}

        bq_schema = columns

        parse = {
            "BIGNUMERIC": "decimal(38, 9)",
            "FLOAT": "double",
            "INTEGER": "long",
            "DATETIME": "timestamp",
            "STRING": "string",
            "TIMESTAMP": "timestamp",
            "DATE": "date",
            "BOOLEAN": "boolean",
            "NUMERIC": "decimal(38, 9)",
        }

        def __bq_to_spark(schema: list, root: dict):
            for column in schema:
                if column["type"] == "RECORD":
                    if column["mode"] == "NULLABLE":
                        if "list" in [
                            field["name"] for field in column["record_fields"]
                        ]:

                            spark_column = {
                                "metadata": {},
                                "name": column["name"],
                                "nullable": False,
                                "type": {
                                    "containsNull": True,
                                    "elementType": {"fields": [], "type": "struct"},
                                    "type": "array",
                                },
                            }

                            root["fields"].append(spark_column)

                            __bq_to_spark(
                                column["record_fields"][0]["record_fields"][0][
                                    "record_fields"
                                ],
                                spark_column["type"]["elementType"],
                            )
                        else:
                            spark_column = {
                                "metadata": {},
                                "name": column["name"],
                                "nullable": column["mode"] == "NULLABLE",
                                "type": "struct",
                                "fields": [],
                            }

                            __bq_to_spark(column["fields"], spark_column)
                    else:
                        spark_column = {
                            "metadata": {},
                            "name": column["name"],
                            "nullable": column["mode"] == "NULLABLE",
                            "type": "array",
                        }

                        root["fields"].append(spark_column)
                else:
                    spark_column = {
                        "metadata": {},
                        "name": column["name"],
                        "nullable": column["mode"] == "NULLABLE",
                        "type": parse[column["type"]],
                    }

                    root["fields"].append(spark_column)

            return root

        spark_schema = __bq_to_spark(schema=bq_schema, root=root_spark_schema)

        spark_schema2 = StructType.fromJson(spark_schema)

        return spark_schema2

In [50]:
schema_parsed = _cast_json(df, columns)

In [40]:
schema_parsed["uf"].dataType

StringType()

In [51]:
for column in df.schema:
    df = df.withColumn(
        column.name,
        col(column.name).cast(schema_parsed[column.name].dataType)
    )

In [52]:
df.write.parquet("estban_cleared")

23/04/02 22:23:43 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 53, schema size: 65
CSV file: file:///Users/flavio.pangracio/www/tcc/estban_csv/202210_ESTBAN_AG.CSV
23/04/02 22:23:45 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 53, schema size: 65
CSV file: file:///Users/flavio.pangracio/www/tcc/estban_csv/202209_ESTBAN_AG.CSV
23/04/02 22:23:45 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 53, schema size: 65
CSV file: file:///Users/flavio.pangracio/www/tcc/estban_csv/202211_ESTBAN_AG.CSV


23/04/02 22:23:46 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 53, schema size: 65
CSV file: file:///Users/flavio.pangracio/www/tcc/estban_csv/202207_ESTBAN_AG.CSV
23/04/02 22:23:46 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 53, schema size: 65
CSV file: file:///Users/flavio.pangracio/www/tcc/estban_csv/202208_ESTBAN_AG.CSV
23/04/02 22:23:47 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 53, schema size: 65
CSV file: file:///Users/flavio.pangracio/www/tcc/estban_csv/202212_ESTBAN_AG.CSV
